# **第8章 使用schedule定時執行Python腳本**

## 8.1 安裝和配置schedule

### 8.1.1 安裝schedule庫

In [ ]:
pip install schedule

### 8.1.2 基本配置

In [ ]:
import schedule  # 匯入排程模塊，方便管理定時任務
import time  # 匯入時間模塊，用於暫停執行程序

def job():
    # 定義一個名為 job 的函數，這個函數的主要工作是打印訊息。此函數將會是我們定時執行的任務。
    print("每分鐘執行一次這段代碼。")

# 使用 schedule 模塊來設置定時任務
# 每1分鐘調用一次 job 函數。'.minutes' 表示以分鐘為單位，'.do(job)' 表示要執行的函數是 job
schedule.every(1).minutes.do(job)

# 無限迴圈，使程序持續運行以便能夠運行排程任務
while True:
    # 這個方法會檢查所有已經設置的定時任務，如果時間到達，則執行相應的任務
    schedule.run_pending()
    # 暫停1秒，以避免佔用過多的CPU資源
    time.sleep(1)

## 8.2 定時任務設置和管理

### 8.2.1 設置定時任務

In [ ]:
import schedule
import time

def daily_job():
    print("每天早上8點執行這段代碼。")

# 設置每天早上8點執行daily_job函數
schedule.every().day.at("08:00").do(daily_job)

while True:
    schedule.run_pending()
    time.sleep(1)

### 8.2.2 移除和取消定時任務

In [ ]:
import schedule
import time

def job():
    print("這是一個將被取消的定時任務。")

# 使用 schedule 庫設置一個每10秒鐘執行一次的任務
# 將這個任務保存在變量 job_to_cancel 中，以便後續允許取消這個任務
job_to_cancel = schedule.every(10).seconds.do(job)

# 使用 time 模組讓程序休眠10秒
# 這樣在這段時間內，job 函數將至少執行一次
time.sleep(10)

# 取消之前設置的定時任務
# 使用 schedule 庫的 cancel_job 方法，並傳入之前保存的 job_to_cancel 變量
schedule.cancel_job(job_to_cancel)

while True:
    schedule.run_pending()
    time.sleep(1)

In [ ]:
import schedule
import time

def job1():
    print("任務1每10秒執行一次。")

def job2():
    print("任務2每15秒執行一次。")

# 設置兩個定時任務
schedule.every(10).seconds.do(job1)
schedule.every(15).seconds.do(job2)

# 列出所有定時任務
print(schedule.jobs)

# 移除所有定時任務
schedule.clear()

while True:
    schedule.run_pending()
    time.sleep(1)

## 8.3 高級用法

### 8.3.1 結合其他庫使用：pandas、matplotlib和openpyxl

#### 使用pandas處理數據

In [ ]:
import pandas as pd
import random
from datetime import datetime, timedelta

random.seed(42)

# 生成銀行業務的虛擬數據
data = {
    "日期": [datetime.now() - timedelta(days=i) for i in range(10)],
    "交易額": [round(random.uniform(100, 1000), 2) for _ in range(10)],
    "客戶ID": [f"Cust{random.randint(1000, 9999)}" for _ in range(10)]
}

df = pd.DataFrame(data)
print(df)


In [ ]:
# 示例：計算每個客戶的總交易額
grouped_df = df.groupby("客戶ID")["交易額"].sum().reset_index()
print(grouped_df)

#### 使用matplotlib進行數據可視化

In [ ]:
!wget -O TaipeiSansTCBeta-Regular.ttf https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download
import matplotlib
matplotlib.font_manager.fontManager.addfont('TaipeiSansTCBeta-Regular.ttf')
matplotlib.rc('font', family='Taipei Sans TC Beta')

In [ ]:
import matplotlib.pyplot as plt

# 創建柱狀圖
plt.figure(figsize=(10, 5))
plt.bar(grouped_df['客戶ID'], grouped_df['交易額'])
plt.xlabel("客戶ID")
plt.ylabel("總交易額")
plt.title("每個客戶的總交易額")
plt.show()

#### 使用openpyxl將處理結果生成Excel檔案

In [ ]:
from openpyxl import Workbook
from openpyxl.drawing.image import Image
from openpyxl.utils.dataframe import dataframe_to_rows

# 創建Excel文件
wb = Workbook()
ws = wb.active
ws.title = "銀行交易報告"

# 將數據框中的數據寫入Excel
for r in dataframe_to_rows(grouped_df, index=True, header=True):
    ws.append(r)

# 保存Excel文件
wb.save("銀行交易報告.xlsx")

# 將圖表保存為圖片並插入Excel
plt.figure(figsize=(10, 5))
plt.bar(grouped_df['客戶ID'], grouped_df['交易額'])
plt.xlabel("客戶ID")
plt.ylabel("總交易額")
plt.title("每個客戶的總交易額")
plt.savefig("交易額圖表.png")
img = Image("交易額圖表.png")
ws.add_image(img, 'E1')

# 再次保存Excel文件
wb.save("銀行交易報告.xlsx")

## 8.4 案例：自動數據分析和報告生成系統

#### 定期讀取數據並進行處理

In [ ]:
import schedule
import pandas as pd
import matplotlib.pyplot as plt
from openpyxl import Workbook
from openpyxl.drawing.image import Image
from openpyxl.utils.dataframe import dataframe_to_rows
import time
from datetime import datetime, timedelta
import random

# 設置隨機數生成器的種子，以保持結果的可重複性
random.seed(42)

def generate_data():
    """
    生成模擬的交易數據。
    創建一個包含日期、交易額和客戶ID的數據幀。

    返回:
        df (pd.DataFrame): 包含模擬的銀行交易數據的數據幀。
    """
    # 生成樣本數據，包括日期、交易額和客戶ID
    data = {
        "日期": [datetime.now() - timedelta(days=i) for i in range(10)],
        "交易額": [round(random.uniform(100, 1000), 2) for _ in range(10)],
        "客戶ID": [f"Cust{random.randint(1000, 9999)}" for _ in range(10)]
    }
    # 將數據轉換為DataFrame
    df = pd.DataFrame(data)
    return df

def analyze_data(df):
    """
    分析交易數據，按客戶ID分組並計算各客戶的總交易額。

    參數:
        df (pd.DataFrame): 包含原始交易數據的數據幀。

    返回:
        grouped_df (pd.DataFrame): 按客戶分組並彙總了交易額的數據幀。
    """
    # 按客戶ID分組並計算每個客戶的總交易額
    grouped_df = df.groupby("客戶ID")["交易額"].sum().reset_index()
    return grouped_df

def visualize_data(grouped_df):
    """
    可視化每個客戶的總交易額，並保存圖表為圖片文件。

    參數:
        grouped_df (pd.DataFrame): 按客戶分組並彙總了交易額的數據幀。
    """
    # 設置圖表大小並繪制柱狀圖
    plt.figure(figsize=(10, 5))
    plt.bar(grouped_df['客戶ID'], grouped_df['交易額'])
    plt.xlabel("客戶ID")
    plt.ylabel("總交易額")
    plt.title("每個客戶的總交易額")
    # 保存圖表為文件
    plt.savefig("交易額圖表.png")
    plt.close()

def save_to_excel(grouped_df):
    """
    將分析結果保存到Excel文件，並將生成的交易額圖表插入Excel中。

    參數:
        grouped_df (pd.DataFrame): 按客戶分組並彙總了交易額的數據幀。
    """
    # 創建一個新的Excel工作簿和工作表
    wb = Workbook()
    ws = wb.active
    ws.title = "銀行交易報告"

    # 將DataFrame的每一行寫入工作表
    for r in dataframe_to_rows(grouped_df, index=True, header=True):
        ws.append(r)

    # 插入圖表圖片到Excel中
    img = Image("交易額圖表.png")
    ws.add_image(img, 'E1')

    # 保存工作簿
    wb.save("銀行交易報告.xlsx")

def daily_task():
    """
    每日任務：生成模擬數據、分析數據、可視化數據並保存為Excel報告。
    """
    # 生成數據
    df = generate_data()
    # 分析數據
    grouped_df = analyze_data(df)
    # 可視化數據
    visualize_data(grouped_df)
    # 保存分析結果到Excel
    save_to_excel(grouped_df)
    # 打印完成信息
    print("自動數據分析和報告生成完成。")

# 設置每日執行任務的時間
schedule.every().day.at("08:00").do(daily_task)

# 保持腳本運行並等待任務調度
while True:
    schedule.run_pending()
    time.sleep(1)

### 檢查任務執行情況並進行調整

In [ ]:
# 改為每周一早上8點執行一次
schedule.every().monday.at("08:00").do(daily_task)